## Introduction

在之前的教程里，我们介绍了jax基本语法框架，jax高级自动微分功能(advanced automatic differentiation)并展示了如何用jax搭建一个toy的神经网络框架。
在这部分里, 我们尝试用jax搭建一个用于mnist数据集多分类的MLP神经网络模型。具体而言，我们先用熟悉的pytorch的dataloader模块来加载mnist数据集，基于该数据集我们用jax搭建一个MLP模型，对其进行预测及可视化分析。我们之所以在这里使用pytorch的模块，是因为jax的设计与开发人员认为现有的如tensorflow或pytorch自带的dataloader框架已经很成熟了，没有必要在jax里重新造轮子。

   1. 通过pytorch的dataloader模块加载mnist数据集，并转成jax可以处理的类型。
   2. 设计与搭建MLP网络框架与预测模块。
   3. 增加训练模块(training loop)，损失函数(loss function)等。

In [33]:
import numpy as np
import functools

import jax
import jax.numpy as jnp

from jax import grad, jit, vmap, pmap, random

import matplotlib.pyplot as plt
from copy import deepcopy
from typing import Tuple, NamedTuple

好了，我们实现了第一个基于jax的神经网络模型(虽然很简单)。

## MNIST数据集
mnist是计算机视觉(Compute vision)一个经典的新手入门的数据集:
   1. 该数据集包含60,000 个训练图片样本与标签, 10,000个测试图片样本与标签。
   2. 每张图片的大小是($28\times28$),也就是每张图片包含$28\times28=784$向素点
在MNIST数据集任务中，我们的目标是给定一个图片，我们预测其是数字[0,1,2,3,4,5,6,7,8,9]中的哪一类,即多分类问题(10个类别)。

在pytorch中， 我们可以使用torchvision工具集直接下载:

In [25]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

def custom_transform(x):
    return np.ravel(np.array(x, dtype=np.float32))

#训练集
train_dataset = MNIST('train_mnist',train=True,download=True,transform=None)
#测试集
test_dataset = MNIST('test_mnist', train=False,download=True,transform=None)

100%|██████████| 9912422/9912422 [00:00<00:00, 121529806.07it/s]

Extracting train_mnist/MNIST/raw/train-images-idx3-ubyte.gz to train_mnist/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 3679252.03it/s]


Extracting train_mnist/MNIST/raw/train-labels-idx1-ubyte.gz to train_mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 33690368.17it/s]

...

Extracting test_mnist/MNIST/raw/train-labels-idx1-ubyte.gz to test_mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 32014458.56it/s]


Extracting test_mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to test_mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 11587912.88it/s]

Extracting test_mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to test_mnist/MNIST/raw



In [26]:
print(type(train_dataset))

<class 'torchvision.datasets.mnist.MNIST'>


In [27]:
train0 = train_dataset[0]
train_dataset[0][0]

In [24]:
train_dataset[0][1]

5

我们可以看到原始数据里的类型是torchvision.datasets, 我们现在要基于jax的预测模型，我们需要对其进行转换，我们可以用其内置的transform接口, 先将其转成numpy array的形式:

In [25]:
# np.ravel: flatten the image 将28x28转成(784,)的一维数据
def custom_transform(x):
    return np.ravel(np.array(x, dtype=np.float32))

#训练集
train_dataset = MNIST('train_mnist',train=True,download=True,transform=custom_transform)
#测试集
test_dataset = MNIST('test_mnist', train=False,download=True,transform=custom_transform)

上述中，我们通过transform实现了两个功能:
   1. 通过np.array将原始pytorch的格式转成了jax可处理的numpy.array的格式。
   2. 通过np.ravel将原始28x28的图片转成(784,)的格式。

我们接下用torch里的dataloader来构造神经网络训练的数据流, 这里我们设置batch_size=128,

In [28]:
batch_size=128
train_loader = DataLoader(train_dataset, batch_size, shuffle=True, collate_fn=None, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False, collate_fn=None, drop_last=True)

通过如下代码对数据进行测试，

In [28]:
batch_data = next(iter(train_loader))
print(batch_data)
type(batch_data[0])

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([4, 9, 1, 6, 4, 3, 7, 9, 8, 8, 4, 6, 2, 6, 8, 8, 5, 6, 0, 3, 4, 4, 5, 5,
        7, 9, 8, 0, 3, 4, 8, 1, 9, 4, 3, 3, 3, 3, 2, 9, 4, 0, 1, 9, 6, 0, 8, 0,
        2, 1, 7, 5, 6, 5, 0, 1, 8, 7, 7, 7, 2, 0, 7, 9, 4, 1, 9, 8, 9, 4, 9, 5,
        2, 9, 8, 2, 5, 9, 6, 4, 4, 1, 9, 2, 6, 3, 1, 2, 5, 4, 1, 2, 5, 8, 5, 1,
        6, 2, 7, 9, 6, 6, 5, 5, 5, 5, 8, 2, 8, 9, 0, 1, 9, 6, 5, 4, 7, 4, 4, 7,
        2, 8, 6, 3, 7, 3, 2, 0])]


torch.Tensor

可以看到，上述数据的格式还是pytorch.tensor的格式(左边是128*784的torch.tensor, 右边是label:长度为128的torch.tensor)。可以通过Dataloader中的collate_fn参数接口对其进行转换:

In [29]:
def custom_collate_fn(batch):
    transposed_data = list(zip(*batch))

    labels = np.array(transposed_data[1])
    imgs = np.stack(transposed_data[0])

    return imgs, labels

train_loader = DataLoader(train_dataset, batch_size, shuffle=True, collate_fn=custom_collate_fn, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False, collate_fn=custom_collate_fn, drop_last=True)

In [30]:
batch_data = next(iter(train_loader))
print(batch_data)
type(batch_data[0])

(array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ...,

numpy.ndarray

## Jax MLP模型框架
我们现在基于之前jax与多层感知机(MLP:multi-layer perceptron)搭建一个MLP模型框架。对于mnist,我们现有的每张图片的输入维度是$1\times 784$,我们要预测是10个类型的概率，相应地，我们可以搭建一个层数分别为[784,512,256,10]的MLP模型框架:

In [41]:
def init_MLP(layer_widths):
    params = []
    for in_width, out_width in zip(layer_widths[:-1], layer_widths[1:]):
        params.append([
            np.random.randn(out_width, in_width),# the weight
            np.random.randn(out_width,) # the bias
        ])
    return params

MLP_params = init_MLP([784, 512, 256, 10])

对于设计好的参数，我们可以用之前学习过的tree_map函数打印；确认该函数框架和我们希望的是一致的

In [32]:
print(jax.tree_map(lambda x: x.shape, MLP_params))

[[(512, 784), (512,)], [(256, 512), (256,)], [(10, 256), (10,)]]


对于该框架，存在着一些问题: 我们通过np.random.randn的模型参数默认是N(0,1)，对于如此大的参数，我们通过backpropagation可能会遇到梯度爆炸(Gradient explosion)等问题:
   1. 在开始尝试新的高大上的初始始化方法前，我们简单对其进行缩放(*0.1, *0.01)等。
   2. 基于先前对随机数生成机制的介绍，对于神经网络模型的函数或类:我们需要显式地赋予一个状态变量。对于一个MLP模型，理论上，每一层layer我们都需要显式地赋予一个状态变量，我们可以用之前介绍的random.split的机制实现。

In [43]:
def init_MLP(layer_widths,parent_key, scale=0.01):
    params = []
    keys = jax.random.split(parent_key, num=len(layer_widths)-1)
    for in_width, out_width,key in zip(layer_widths[:-1], layer_widths[1:],keys):
        weight_key, bias_key = jax.random.split(key)
        params.append([
            scale*jax.random.normal(weight_key, shape=(out_width, in_width)),# the weight
            scale*jax.random.normal(bias_key, shape=(out_width,)) # the bias
        ])
    return params

seed=0
key = jax.random.PRNGKey(seed)
MLP_params = init_MLP([784, 512, 256, 10],key)

### 预测模块:交叉熵损失与log-sum-exp trick
我们接下来跟进预测模块的代码, 在开始前我们介绍一下机器学习分类任务常见的log-sum-exp trick。
对于神经网络解决多分类问题(假设我们有K个类别), 一种常用的方式是在最后一层通过softmax将预测的K个类别的值$x_j$映射至$[0,1]$区间,用公式表示:
$$
Pr(G=j|X)=\frac{exp(x_j)}{\sum\limits_{i=1}^K exp(x_i)}
$$
对于分类问题，常用的loss function是交叉熵损失(cross-entropy loss),即对上述每个预测的概率取对数，即有:
$$
log(\frac{exp(x_j)}{\sum\limits_{i=1}^K exp(x_i)}) =  log(exp(x_j)) - log(\sum\limits_{i=1}^K exp(x_i)) = x_j-log(\sum\limits_{i=1}^K exp(x_i))
$$
也就对于cross-entropy loss,我们可以直接用每个类别中间预测值$x_j$减去其所有的类别array的log-sum-exp的映射结果。
在实现中，对于log-sum-exp操作，scipy.special里调用logsumexp函数，在jax里,

In [34]:
# from scipy.special import logsumexp
from jax.scipy.special import logsumexp
def MLP_predict(params, x):
    hidden_layers = params[:-1]
    activation = x
    for w,b in hidden_layers:
        activation = jax.nn.relu(jnp.dot(w,activation)+b)

    #在最后一层，我们不想使用relu激活函数
    w_last,b_last = params[-1]
    logits = jnp.dot(w_last, activation)+b_last
    
    return  logits - logsumexp(logits)

随机生成一个图片的shape的随机样本对上述进行测试：

In [35]:
# test single example
mnist_img_size = (28, 28)
dummy_img_flat = np.random.randn(np.prod(mnist_img_size))
print(dummy_img_flat.shape)

prediction = MLP_predict(MLP_params, dummy_img_flat)
print(prediction.shape)

(784,)
(10,)


同时对16张(一个batch里的)图片进行预测测试:

In [36]:
# test batched function
batched_MLP_predict = vmap(MLP_predict, in_axes=(None, 0))

dummy_imgs_flat = np.random.randn(16, np.prod(mnist_img_size))
print(dummy_imgs_flat.shape)
predictions = batched_MLP_predict(MLP_params, dummy_imgs_flat)
print(predictions.shape)

(16, 784)
(16, 10)


#### training loop and evaluation
在开始这部分之前，我们介绍一下one-hot-embedding, 对于mnist数据集，假设我们有如下5个label向量:[0,2,4,6,8],我们可以将其表示成5*10(10表示类别总数),其one-hot embedding可以表示如下:

$$label = 
\begin{bmatrix}
\begin{array}{c}
    0  \\
    2 \\
    4\\
    6\\
    8 
\end{array}
\end{bmatrix} \Longrightarrow 
\begin{bmatrix}
\begin{array}{cccccccccc}
     1 & 0& 0 &0 & 0 & 0 & 0 &0 & 0 & 0  \\
     0 & 0& 1 &0 & 0 & 0 & 0 &0 & 0 & 0  \\
     0 & 0& 0 &0 & 1 & 0 & 0 &0 & 0 & 0 \\
     0 & 0& 0 &0 & 0 & 0 & 1 &0 & 0 & 0 \\
     0 & 0& 0 &0 & 0 & 0 & 0 &0 & 1 & 0  
\end{array}
\end{bmatrix}
$$

也就是若其第label为k, 其第k列下标为1，其余都为0.
在jax中，我们可以通过 jax.nn.one_hot实现one-hot embedding。相应地，我们可以实现一个简单的基于cross-entropy loss的神经网络训练loop:

In [31]:
# transfer the data to jnp
# optimization - loading the whole dataset into memory
train_images = jnp.array(train_dataset.data).reshape(len(train_dataset), -1)
train_lbls = jnp.array(train_dataset.targets)

test_images = jnp.array(test_dataset.data).reshape(len(test_dataset), -1)
test_lbls = jnp.array(test_dataset.targets)

#### 损失函数(loss function)与准确度(accuracy)：

In [39]:
def loss_fn(params, imgs, gt_lbls):
    predictions = batched_MLP_predict(params, imgs)

    return -jnp.mean(predictions * gt_lbls)

def accuracy(params, dataset_imgs, dataset_lbls):
    pred_classes = jnp.argmax(batched_MLP_predict(params, dataset_imgs), axis=1)
    return jnp.mean(dataset_lbls == pred_classes)

Training loop,我们这里使用jax中的value_and_grad同时输入函数的loss function与gradient.

In [1]:
@jit
def update(params, imgs, gt_lbls, lr=0.01):
    loss, grads = jax.value_and_grad(loss_fn)(params, imgs, gt_lbls)
    return loss, jax.tree_map(lambda p, g: p - lr*g, params, grads)

num_epochs = 10
# Create a MLP
MLP_params = init_MLP([np.prod(mnist_img_size), 512, 256, len(MNIST.classes)], key)

for epoch in range(num_epochs):
    for cnt, (imgs, lbls) in enumerate(train_loader):
        gt_labels = jax.nn.one_hot(lbls, len(MNIST.classes))
        
        loss, MLP_params = update(MLP_params, imgs, gt_labels)
        
        if cnt % 50 == 0:
            print(loss)

    print(f'Epoch {epoch}, train acc = {accuracy(MLP_params, train_images, train_lbls)} test acc = {accuracy(MLP_params, test_images, test_lbls)}')

NameError: name 'jit' is not defined

In [ ]:
imgs, lbls = next(iter(test_loader))
img = imgs[0].reshape(mnist_img_size)
gt_lbl = lbls[0]
print(img.shape)

import matplotlib.pyplot as plt

pred = jnp.argmax(MLP_predict(MLP_params, np.ravel(img)))
print('pred', pred)
print('gt', gt_lbl)

plt.imshow(img); plt.show()